In [2]:
!pip install pydub

In [3]:
!pip install pyrubberband

In [4]:
!pip install librosa

In [5]:
!sudo apt-get update
!sudo apt-get install rubberband-cli

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,853 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,161 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

In [6]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# main mixing - bpm + dastgah + rand volumn change
import os
from pydub import AudioSegment
import random
import re
import librosa
import pyrubberband as pyrb
import numpy as np

small_mixings = [['tar', 'tonbak'], ['sitar', 'tonbak'], ['kamancheh', 'tonbak'], ['ney', 'tonbak'], ['santur', 'tonbak'], ['violin', 'tonbak'], ['piano', 'tonbak'],
                 ['violin', 'tar', 'tonbak'], ['piano', 'violin', 'tonbak']]
medium_mixings = [['tar', 'kamancheh', 'santur', 'tonbak'], ['ney', 'tar', 'tonbak', 'daaf'], ['kamanched', 'santur', 'tonbak'], ['sitar', 'tar', 'tonbak', 'daaf'],
                  ['tar', 'kamancheh', 'santur', 'tonbak', 'violin'], ['piano', 'violin', 'tar', 'tonbak']]
orchestral_mixings = [['santur', 'tar', 'sitar', 'tonbak', 'daaf'], ['santur', 'tar', 'kamanched', 'tonbak', 'daaf'], ['santur', 'tar', 'ney', 'tonbak', 'daaf'],
                      ['santur', 'tar', 'sitar', 'tonbak', 'daaf', 'violin'], ['santur', 'tar', 'kamanched', 'tonbak', 'daaf', 'violin'],
                      ['santur', 'tar', 'ney', 'tonbak', 'daaf', 'violin']]
with_avaz = [['avaz', 'tonbak', 'tar'], ['avaz', 'tar'], ['avaz', 'tonbak'], ['avaz', 'sitar'], ['avaz', 'sitar', 'daaf'], ['avaz', 'ney', 'tonbak'], ['avaz', 'ney'],
             ['avaz', 'violin', 'tonbak'], ['avaz', 'piano', 'tonbak'], ['avaz', 'violin'], ['avaz', 'piano']]

main_instruments = ['kamancheh', 'ney', 'santur', 'sitar', 'tar', 'avaz', 'piano', 'violin']
percussion_instruments = ['tonbak', 'daaf']
dastgahs = ['shur', 'chahargah', 'homayoon', 'mahur', 'nava', 'rast-panjgah', 'segah']



def adjust_bpm(audio_segment, original_bpm, target_bpm):
    """Adjust BPM of an AudioSegment using rubberband"""
    if original_bpm == target_bpm:
        return audio_segment

    # Convert AudioSegment to raw audio data
    samples = audio_segment.get_array_of_samples()
    # Convert array.array to numpy array
    samples_np = np.array(samples)

    sr = audio_segment.frame_rate
    stretch_factor = original_bpm / target_bpm

    # Apply time stretching
    stretched_samples = pyrb.time_stretch(samples_np, sr, stretch_factor)

    # Create new AudioSegment
    return AudioSegment(
        stretched_samples.tobytes(),
        frame_rate=sr,
        sample_width=audio_segment.sample_width,
        channels=audio_segment.channels
    )

def get_bpm_from_mp3(mp3_path):
    try:
        # Load the audio file
        y, sr = librosa.load(mp3_path)

        # Compute the tempo
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

        return tempo
    except Exception as e:
        print(f"Error processing {mp3_path}: {str(e)}")
        return None

# Configuration
input_base_dir = "/content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset"
output_dir = "/content/drive/MyDrive/research/MusicGeneration/multi_instrument_dataset/data"
num_samples = 19683
target_duration_ms = 5000  # 5 seconds
max_instruments = 5
min_instruments = 1

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Get all instruments and their dastgahs
instruments = [d for d in os.listdir(input_base_dir)
               if os.path.isdir(os.path.join(input_base_dir, d))]
dastgahs = ['shur', 'chahargah', 'homayoon', 'mahur', 'nava', 'rast-panjgah', 'segah']

# Track seen label combinations to avoid duplicates
seen_combinations = set()

# Open metadata file
with open(os.path.join(output_dir, "dataset_description.txt"), 'a+') as meta_file:
    meta_file.write("file_name,labels\n")  # Header

    j = 0  # Sample counter
    attempts = 0
    max_attempts = num_samples * 10  # To avoid infinite loop

    while j < num_samples and attempts < max_attempts:
        attempts += 1
        selected_dastgah = random.choice(dastgahs)

        # Randomly select instruments and dastgahs
        if j >= 40000:
            inst_count = random.randint(1, len(instruments))
            selected_instruments = random.sample(instruments, inst_count)
        elif j >= 30000:
            selected_instruments = random.sample(with_avaz, 1)[0]
        elif j >= 20000:
            selected_instruments = random.sample(orchestral_mixings, 1)[0]
        elif j >= 10000:
            selected_instruments = random.sample(medium_mixings, 1)[0]
        else:
            selected_instruments = random.sample(small_mixings, 1)[0]

        audio_mix = None
        instrument_labels = []
        selected_audios = []
        bpms = []


        for instrument in selected_instruments:
            if instrument in ['kamancheh', 'ney', 'santur', 'sitar', 'tar', 'avaz', 'piano', 'violin']:
                dir_path = os.path.join("/content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset", instrument, selected_dastgah)
            else:
                dir_path = os.path.join("/content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset", instrument)
            try:
                sample_file = random.choice(os.listdir(dir_path))
                audio_path = os.path.join(dir_path, sample_file)
                sound = AudioSegment.from_file(audio_path)
                sound += random.randint(0, 5)
                selected_audios.append(sound)
                bpms.append(get_bpm_from_mp3(audio_path))

            except Exception as e:
                print(f"Could not decode {audio_path}: {e}")
                continue

            # Extract sample index from filename (e.g., sample3.mp3 → 3)
            match = re.search(r'sample(\d+)\.mp3', sample_file)
            sample_index = match.group(1) if match else "x"

            if instrument in ['kamancheh', 'ney', 'santur', 'sitar', 'tar', 'avaz', 'piano', 'violin']:
                # Add to labels
                instrument_labels.append(f"{instrument}-{selected_dastgah}-{sample_index}")
            else:
                instrument_labels.append(f"{instrument}-{sample_index}")

        # Select target BPM (use the most common BPM or random from available)
        if bpms:
          target_bpm = random.choice([bpm for bpm in bpms if bpm is not None])
        else:
          continue # Skip if no valid BPMs found

        # Adjust all audio to target BPM
        adjusted_audio = []
        for i, audio in enumerate(selected_audios):
            try:
                # Handle cases where BPM could not be determined
                if bpms[i] is not None and target_bpm is not None:
                    adjusted = adjust_bpm(audio, bpms[i], target_bpm)
                    adjusted_audio.append(adjusted)
                else:
                    adjusted_audio.append(audio)  # Use original if BPM is None
            except Exception as e:
                print(f"Error adjusting BPM: {e}")
                adjusted_audio.append(audio)  # Use original if adjustment fails


        if not instrument_labels or not adjusted_audio:
            continue

        # Mix the audio
        output = adjusted_audio[0]
        for i in range(1, len(adjusted_audio)):
            output = output.overlay(adjusted_audio[i])

        # Sort labels to normalize and check for duplicates
        label_key = tuple(sorted(instrument_labels))
        if label_key in seen_combinations:
            continue

        seen_combinations.add(label_key)

        # Save the mixed audio
        output_path = os.path.join(output_dir, f"sample{j}.mp3")
        output.export(output_path, format="mp3")

        # Write metadata
        meta_file.write(f"data/sample{j}.mp3,{ ' '.join(instrument_labels) }\n")

        if j % 10 == 0:
            print(f"Created {j} samples")

        j += 1

print(f"Finished creating {j} unique polyphonic samples in {output_dir}")

Created 44120 samples


/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1345
  warnings.warn(


Error adjusting BPM: rate must be strictly positive
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/tonbak/sample_162.wav: Cannot choose from an empty sequence
Created 44130 samples
Created 44140 samples
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/sitar/rast-panjgah/sample235.mp3: Cannot choose from an empty sequence
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/santur/rast-panjgah/sample177.mp3: Cannot choose from an empty sequence
Created 44150 samples
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/violin/chahargah/sample220.mp3: Cannot choose from an empty sequence
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/tonbak/sample_17.wav: Cannot choose from an empty sequence
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/piano/nava/sample77.mp3: Cannot choose from an 

/tmp/ipython-input-1861551589.py:37: RuntimeWarning: divide by zero encountered in divide
  stretch_factor = original_bpm / target_bpm
/tmp/ipython-input-1861551589.py:37: RuntimeWarning: divide by zero encountered in divide
  stretch_factor = original_bpm / target_bpm
/tmp/ipython-input-1861551589.py:37: RuntimeWarning: divide by zero encountered in divide
  stretch_factor = original_bpm / target_bpm
/tmp/ipython-input-1861551589.py:37: RuntimeWarning: divide by zero encountered in divide
  stretch_factor = original_bpm / target_bpm
/tmp/ipython-input-1861551589.py:37: RuntimeWarning: divide by zero encountered in divide
  stretch_factor = original_bpm / target_bpm


Created 44230 samples
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/violin/rast-panjgah/sample51.mp3: Cannot choose from an empty sequence
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/kamancheh/rast-panjgah/sample46.mp3: Cannot choose from an empty sequence
Created 44240 samples
Error adjusting BPM: rate must be strictly positive
Error adjusting BPM: rate must be strictly positive
Created 44250 samples
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/sitar/shur/sample345.mp3: Cannot choose from an empty sequence
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/sitar/rast-panjgah/sample13.mp3: Cannot choose from an empty sequence
Created 44260 samples
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/tonbak/sample_940.wav: Cannot choose from an empty sequence
Error adjusting BPM: rate must be strictly positiv

...

Created 46770 samples
Created 46780 samples
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/sitar/nava/sample76.mp3: Cannot choose from an empty sequence
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/tonbak/sample_293.wav: Cannot choose from an empty sequence
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/ney/rast-panjgah/sample156.mp3: Cannot choose from an empty sequence
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/daaf/sample_503.wav: Cannot choose from an empty sequence
Created 46790 samples
Created 46800 samples
Created 46810 samples
Created 46820 samples
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/kamancheh/rast-panjgah/sample44.mp3: Cannot choose from an empty sequence
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/tonbak/sample_405.wav: Cannot choose from

/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=816
  warnings.warn(
/tmp/ipython-input-1861551589.py:37: RuntimeWarning: divide by zero encountered in divide
  stretch_factor = original_bpm / target_bpm


Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/santur/rast-panjgah/sample41.mp3: Cannot choose from an empty sequence
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/daaf/sample_562.wav: Cannot choose from an empty sequence
Created 46840 samples


/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=816
  warnings.warn(


Error adjusting BPM: rate must be strictly positive
Created 46850 samples
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/ney/rast-panjgah/sample52.mp3: Cannot choose from an empty sequence
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/daaf/sample_943.wav: Cannot choose from an empty sequence
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/violin/rast-panjgah/sample70.mp3: Cannot choose from an empty sequence
Created 46860 samples
Created 46870 samples
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/tonbak/sample_1312.wav: Cannot choose from an empty sequence
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/ney/mahur/sample104.mp3: Cannot choose from an empty sequence
Created 46880 samples
Error adjusting BPM: rate must be strictly positive
Could not decode /content/drive/MyDrive/research/MusicGeneration/Mon

In [ ]:
# only dastgah

import os
import random
from pydub import AudioSegment
import pyrubberband as pyrb
import re
import librosa


main_instruments = ['kamancheh', 'ney', 'santur', 'sitar', 'tar', 'avaz', 'piano', 'violin']
percussion_instruments = ['tonbak', 'daaf']
dastgahs = ['shur', 'chahargah', 'homayoon', 'mahur', 'nava', 'rast-panjgah', 'segah']
input_base_dir = "/content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset"
output_dir = "/content/drive/MyDrive/research/MusicGeneration/multi_instrument_dataset_dastgahonly/data"
num_samples = 50000
target_duration_ms = 5000  # 5 seconds
max_instruments = 5
min_instruments = 1

# Weighted probabilities for instruments (Avaz has lower probability)
instrument_weights = {
    'kamancheh': 10,
    'ney': 10,
    'santur': 10,
    'sitar': 10,
    'tar': 10,
    'avaz': 3,  # Lower weight for Avaz
    'piano': 10,
    'violin': 10
}

# Track seen label combinations to avoid duplicates
seen_combinations = set()


# Open metadata file
with open(os.path.join(output_dir, "dataset_description.txt"), 'a+') as meta_file:
    meta_file.write("file_name,labels\n")  # Header

    j = 43081  # Sample counter
    attempts = 0
    max_attempts = num_samples * 10  # To avoid infinite loop

    while j < num_samples and attempts < max_attempts:
        selected_dastgah = random.choice(dastgahs)
        attempts += 1

        # Randomly select instruments and dastgahs
        main_inst_count = random.randint(0, 5)
        main_instruments_selected = random.choices(
            main_instruments,
            weights=[instrument_weights[inst] for inst in main_instruments],
            k=main_inst_count
        )
        percuss_count = random.randint(0, 2)
        percussion_instruments_selected = random.sample(percussion_instruments, percuss_count)

        selected_instruments = percussion_instruments_selected + main_instruments_selected

        audio_mix = None
        instrument_labels = []

        for instrument in selected_instruments:
            if instrument in ['kamancheh', 'ney', 'santur', 'sitar', 'tar', 'avaz', 'piano', 'violin']:
                dir_path = os.path.join("/content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset", instrument, selected_dastgah)
            else:
                dir_path = os.path.join("/content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset", instrument)
            try:
                sample_file = random.choice(os.listdir(dir_path))
                audio_path = os.path.join(dir_path, sample_file)
                sound = AudioSegment.from_file(audio_path)
            except Exception as e:
                print(f"Could not decode {audio_path}: {e}")
                continue

            # Mix with other tracks
            if audio_mix is None:
                audio_mix = sound
            else:
                audio_mix = audio_mix.overlay(sound)

            # Extract sample index from filename (e.g., sample3.mp3 → 3)
            match = re.search(r'sample(\d+)\.mp3', sample_file)
            sample_index = match.group(1) if match else "x"

            if instrument in ['kamancheh', 'ney', 'santur', 'sitar', 'tar', 'avaz', 'piano', 'violin']:
                # Add to labels
                instrument_labels.append(f"{instrument}-{selected_dastgah}-{sample_index}")
            else:
                instrument_labels.append(f"{instrument}-{sample_index}")

        if not instrument_labels or audio_mix is None:
            continue

        # Sort labels to normalize and check for duplicates
        label_key = tuple(sorted(instrument_labels))
        if label_key in seen_combinations:
            continue

        seen_combinations.add(label_key)

        # Save the mixed audio
        output_path = os.path.join(output_dir, f"sample{j}.mp3")
        audio_mix.export(output_path, format="mp3")

        # Write metadata
        meta_file.write(f"data/sample{j}.mp3,{ ' '.join(instrument_labels) }\n")

        if j % 10 == 0:
            print(f"Created {j} samples")

        j += 1

Created 43090 samples
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/santur/rast-panjgah/sample47.mp3: Cannot choose from an empty sequence
Created 43100 samples
Created 43110 samples
Created 43120 samples
Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/santur/shur/2.mp3: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-li

In [ ]:
# only BPM
import os
from pydub import AudioSegment
import random
import re
import librosa
import pyrubberband as pyrb
import numpy as np


def adjust_bpm(audio_segment, original_bpm, target_bpm):
    """Adjust BPM of an AudioSegment using rubberband"""
    if original_bpm == target_bpm:
        return audio_segment

    # Convert AudioSegment to raw audio data
    samples = audio_segment.get_array_of_samples()
    # Convert array.array to numpy array
    samples_np = np.array(samples)

    sr = audio_segment.frame_rate
    stretch_factor = original_bpm / target_bpm

    # Apply time stretching
    stretched_samples = pyrb.time_stretch(samples_np, sr, stretch_factor)

    # Create new AudioSegment
    return AudioSegment(
        stretched_samples.tobytes(),
        frame_rate=sr,
        sample_width=audio_segment.sample_width,
        channels=audio_segment.channels
    )

def get_bpm_from_mp3(mp3_path):
    try:
        # Load the audio file
        y, sr = librosa.load(mp3_path)

        # Compute the tempo
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

        return tempo
    except Exception as e:
        print(f"Error processing {mp3_path}: {str(e)}")
        return None

# Configuration
input_base_dir = "/content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset"
output_dir = "/content/drive/MyDrive/research/MusicGeneration/multi_instrument_dataset_bpmonly/data"
num_samples = 19683
target_duration_ms = 5000  # 5 seconds
max_instruments = 5
min_instruments = 1

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Get all instruments and their dastgahs
instruments = [d for d in os.listdir(input_base_dir)
               if os.path.isdir(os.path.join(input_base_dir, d))]
dastgahs = ['shur', 'chahargah', 'homayoon', 'mahur', 'nava', 'rast-panjgah', 'segah']

# Track seen label combinations to avoid duplicates
seen_combinations = set()

# Open metadata file
with open(os.path.join(output_dir, "dataset_description.txt"), 'a+') as meta_file:
    meta_file.write("file_name,labels\n")  # Header

    j = 19668  # Sample counter
    attempts = 0
    max_attempts = num_samples * 10  # To avoid infinite loop

    while j < num_samples and attempts < max_attempts:
        attempts += 1

        # Randomly select instruments and dastgahs
        inst_count = random.randint(min_instruments, max_instruments)
        selected_instruments = random.sample(instruments, inst_count)

        audio_mix = None
        instrument_labels = []
        selected_audios = []
        bpms = []


        for instrument in selected_instruments:
            if instrument in ['kamancheh', 'ney', 'santur', 'sitar', 'tar', 'avaz', 'piano', 'violin']:
                dastgah = random.choice(dastgahs)
                dir_path = os.path.join("/content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset", instrument, dastgah)
            else:
                dir_path = os.path.join("/content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset", instrument)
            try:
                sample_file = random.choice(os.listdir(dir_path))
                audio_path = os.path.join(dir_path, sample_file)
                sound = AudioSegment.from_file(audio_path)
                selected_audios.append(sound)
                bpms.append(get_bpm_from_mp3(audio_path))

            except Exception as e:
                print(f"Could not decode {audio_path}: {e}")
                continue

            # Extract sample index from filename (e.g., sample3.mp3 → 3)
            match = re.search(r'sample(\d+)\.mp3', sample_file)
            sample_index = match.group(1) if match else "x"

            if instrument in ['kamancheh', 'ney', 'santur', 'sitar', 'tar', 'avaz', 'piano', 'violin']:
                # Add to labels
                instrument_labels.append(f"{instrument}-{dastgah}-{sample_index}")
            else:
                instrument_labels.append(f"{instrument}-{sample_index}")

        # Select target BPM (use the most common BPM or random from available)
        if bpms:
          target_bpm = random.choice([bpm for bpm in bpms if bpm is not None])
        else:
          continue # Skip if no valid BPMs found

        # Adjust all audio to target BPM
        adjusted_audio = []
        for i, audio in enumerate(selected_audios):
            try:
                # Handle cases where BPM could not be determined
                if bpms[i] is not None and target_bpm is not None:
                    adjusted = adjust_bpm(audio, bpms[i], target_bpm)
                    adjusted_audio.append(adjusted)
                else:
                    adjusted_audio.append(audio)  # Use original if BPM is None
            except Exception as e:
                print(f"Error adjusting BPM: {e}")
                adjusted_audio.append(audio)  # Use original if adjustment fails


        if not instrument_labels or not adjusted_audio:
            continue

        # Mix the audio
        output = adjusted_audio[0]
        for i in range(1, len(adjusted_audio)):
            output = output.overlay(adjusted_audio[i])

        # Sort labels to normalize and check for duplicates
        label_key = tuple(sorted(instrument_labels))
        if label_key in seen_combinations:
            continue

        seen_combinations.add(label_key)

        # Save the mixed audio
        output_path = os.path.join(output_dir, f"sample{j}.mp3")
        output.export(output_path, format="mp3")

        # Write metadata
        meta_file.write(f"data/sample{j}.mp3,{ ' '.join(instrument_labels) }\n")

        if j % 10 == 0:
            print(f"Created {j} samples")

        j += 1

print(f"Finished creating {j} unique polyphonic samples in {output_dir}")

Could not decode /content/drive/MyDrive/research/MusicGeneration/Monophonic_dataset/ney/nava/1.mp3: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-li

In [ ]:
!sudo apt-get update
!sudo apt-get install rubberband-cli

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,202 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,085 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/univer